# Deploying a Python Application to Linux Servers
*Python Flask Tutorial: Deploying Your Application - Deploy to a Linux Server | [Youtube](https://youtu.be/goToXTC96Co)*

<hr>

## Set up a Linux server from scratch

*When selecting server options with cloud services such as AWS, linode, etc.*
1. Create a SSD Linux server
2. Mount an image (*CentOS / Ubuntu / etc.*)
3. Determine a geographic region for the server to be hosted in
4. Wait for server to be spawned successfully


*Accessing/updating the server*
- Using Linux `bash` then use `ssh root@<ip_address>` to access the machine
- `apt update && apt upgrade` to update the machine's software
- Add local IP address to `/etc/hosts`
- Add user to machine with `adduser <username>`
- Add user to `sudo` group: `adduser <username> sudo`
- Log out as `root` and login as `<username>`
- Login with `ssh <username>@<ip_address>`


*Add a public `ssh` key into server*
- `scp ~/.ssh/<ssh_key>.pub <username>@<ip_address>:~/.ssh/authorized_keys`
- Gives read and write permission to authorized SSH keys
    - Read, write and execute for the group: `sudo chmod 700 ~/.ssh/`
    - Read and write for all files: `sudo chmod 600 ~/.ssh/*`


*Deny root login permissions via config file*
- `sudo vim /etc/ssh/sshd_config`
- Change `PermitRootLogin` to `no` once `SSH` access is successful


*Set up a firewall on server*
- `sudo apt install ufw` (*uncomplicated firewall*)
- `sudo ufw default allow outgoing` / `sudo ufw default deny incoming`
- Explicitly allow for specific traffic:
    - `sudo ufw allow ssh`
    - `sudo ufw allow 5000` (*allowing port 5000*)
- `sudo ufw enable` enables the firewall
- `sudo ufw status` shows all allowed traffic


****

## Deploying code to server

*Transfer local to server*
- `scp -r <local_path> <username>@<ip_address>:<server_path>` transfers from local to server

*Create virtual environment on server*
- `sudo apt install python3-pip` installs `pip` on server
- `sudo apt install python3-venv` installs `venv` on server
- `python3 -m venv <project_name>/<env_name>` creates the virtual environment in the project
- `source <env_name>/bin/activate` to activate the virtual environment
- `pip install -r requirements.txt` to install the requirements specified by the project


*Set environment in server based on local environment variables*
- `sudo touch /etc/config.json` to create an environment config file
- Find environment config in local environment using `Python`

```python
# Get local environment variables using Python
>>> import os
>>> os.environ.get('SECRET_KEY')
'<SECRET_KEY>'
>>> os.environ.get('SQLALCHEMY_DATABASE_URI')
'<DATABASE_URI>'
>>> os.environ.get('EMAIL_USER')
'<EMAIL_USER>'
>>> os.environ.get('EMAIL_PASS')
'<EMAIL_PASS>'
```
    
- `sudo vim /etc/config.json` to edit the file

```JSON
# config.json file
{
        "SECRET_KEY": "<SECRET_KEY>",
        "SQLALCHEMY_DATABASE_URI": "<DATABASE_URI>",
        "EMAIL_USER": "<EMAIL_USER>",
        "EMAIL_PASS": "<EMAIL_PASS">
}
```

- Load config file into `config.py`

```python
import os
import json

# Load .json file into a Python dictionary
with open('/etc/config.json') as config_file:
    config = json.load(config_file)
    
# Modify config class to get values from config file
class Config:
    SECRET_KEY = config.get('SECRET_KEY')
    SQLALCHEMY_DATABASE_URI = config.get('SQLALCHEMY_DATABASE_URI')
    EMAIL_USER = config.get('EMAIL_USER')
    EMAIL_PASS = config.get('EMAIL_PASS')
    EMAIL_SERVER = 'smtp.googlemail.com'
    EMAIL_PORT = 587
    EMAIL_USE_TLS = True
```


*Deploy Python Flask app in server*
- `export FLASK_APP=<scriptname>.py` runs `<scriptname>.py` when `flask run` is called
- `flask run --host=0.0.0.0` allows access to the temporary environment from local browser
- Enter `<ip_address>:<port_num>` in browser to see application

****

## Using `nginx` & `gunicorn` to go live with application

*Installation*
- `sudo apt install nginx`
- `pip install gunicorn` within the virtual environment

*Update configuration for `nginx` and `gunicorn`*
- Allow `nginx` to handle web elements
    - `sudo rm /etc/nginx/sites-enabled/default` removes default configuration
    - `sudo vim /etc/nginx/sites-enabled/<app_name>` creates a new config file
    
    ```console
    server {
            listen 80; # port number
            server_name <ip_address>;
            
            location /static {
                    alias <path_to_static_files>;
            }
            
            location / {
                    proxy_pass http://localhost:8000;
                    include /etc/nginx/proxy_params;
                    proxy_redirect off;
            }
    }
    ```
    
    - `sudo ufw allow http/tcp` to allow `http/tcp` traffic
    - `sudo ufw delete allow 5000` to delete development port
    - `sudo vim /etc/nginx/nginx.conf` and set `client_max_body_size` to a larger value
    - `sudo systemctl restart nginx` restarts `nginx` server
    
    
- Allow `gunicorn` to handle `Python` code
    - `gunicorn -w <num_workers> <script_name>:<flask_variable_name>` sets workers to run script
        - $\text{Workers} = (2 \times \text{# of Cores}) + 1$
        - Run `nproc -all` to find number of cores in server
        - In flask, we initialize app creation with `app = create_app()` and therefore `<flask_variable_name>` will be `app` in this case
        - Example: `gunicorn -w 3 run:app`


*Keep application running in server*
- `sudo apt install supervisor`
- `sudo vim /etc/supervisor/conf.d/<appname>.conf`

```console
# Get supervisor to run gunicorn command
[program:<appname>]
directory=<project_pathname>
command=<project_pathname>/venv/bin/gunicorn -w 3 run:app
user=<username>
autostart=true
autorestart=true
stopasgroup=true
killasgroup=true
stderr_logfile=/var/log/<appname>/<appname>.err.log
stdout_logfile=/var/log/<appname>/<appname>.out.log
```

- `sudo mkdir -p /var/log/<appname>`
- `sudo touch /var/log/<appname>.err.log`
- `sudo touch /var/log/<appname>.out.log`
- `sudo supervisorctl reload`


*Run application in browswer*
- Enter `<ip_address>` in browser


****

<div class="alert alert-block alert-success">
<b>Congratulations!</b>

You have successfully deployed an application in the server!
</div>

****

*Useful command snippets*

| Keys / Commands | Descriptions |
| :-------------- | :----------- |
| `apt update && apt upgrade` | Update server's software |
| `hostnamectl set-hostname <target_hostname>` | Set the hostname of the server |
| `scp <local_path> <username>@<ip_address>:<server_path>` | Secure copies a file to a specified server and destination |
| `sudo systemctl restart sshd` | Restarts the server |
| `sudo apt install` | Super user does advanced package tool install |

****

# Basic code
A `minimal, reproducible example`